In [1]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r agg_actions
%store -r calculator
%store -r editors_conflicts

if ('the_page' not in locals() or 
    'agg_actions' not in locals() or 
    'calculator' not in locals() or 
    'editors_conflicts' not in locals()):
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))
    agg_actions = pickle.load(open("data/agg_actions.p",'rb'))
    calculator = pickle.load(open("data/calculator.p",'rb'))
    editors_conflicts = pickle.load(open("data/editors_conflicts.p",'rb'))

### <span style="color:green"> Modules Imported </span>

In [98]:
## Modules Imported ##

# Display
from IPython.display import display, Markdown as md, clear_output, Javascript, HTML
from datetime import datetime, date

# APIs
from external.wikipedia import WikipediaDV, WikipediaAPI

# Load and process data.
import pickle

# Visualization
import qgrid
from visualization.conflicts_listener import ConflictsListener
from visualization.actions_listener import ActionsListener
from visualization.wordcloud_listener import WCListener
from visualization.owned_listener import OwnedListener

from utils.notebooks import get_date_slider_from_datetime

from ipywidgets import interact, Output, widgets, fixed
from ipywidgets.widgets import Dropdown

# Load the variables stored in the last notebook
%store -r the_page
%store -r total_actions
%store -r conflict_calculator
%store -r conflicts_by_editors

# Check them if in the namespace, otherwise load the default data.
if ('the_page' not in locals() or 
    'total_actions' not in locals() or 
    'conflict_calculator' not in locals() or 
    'conflicts_by_editors' not in locals()):
    
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))
    total_actions = pickle.load(open("data/agg_actions.p",'rb'))
    conflict_calculator = pickle.load(open("data/calculator.p",'rb'))
    conflicts_by_editors = pickle.load(open("data/editors_conflicts.p",'rb'))

In [3]:
display(md(f"# ***Page: {the_page['title']}***"))

# ***Page: Random***

---

# A.  Select an editor to analyze their activity in the context.

The table below presents the conflict score and other related  metrics per editor 
(*editor_id* and *editor* column):

- **conflict_n**: the total number of conflicts
- **conflict**: the sum of conflict scores of all actions (without division)
- **actions**: the total number of actions performed by the editor
- **conflict_score**: the sum of conflict scores of all actions divided by the number of elegible actions
- **conflict_ratio**: the count of all conflicts divided by the number of elegible actions

In [49]:
def display_conflict_score(editor_df):
    global listener
        
    listener = ConflictsListener(editor_df)

    metrics = ['Conflict Score', 'Absolute Conflict Score', 
               'Conflict Ratio',  'Number of Conflicts', 
               'Total Elegible Actions']
    #display(md(f'*Total Page conflict score: {calculator.get_page_conflict_score()}*'))
    display(md(f'*Total Page conflict score: {editor_df.conflict.sum() / editor_df.elegibles.sum()}*'))

    # Visualization

    interact(listener.listen,
             #_range = get_date_slider_from_datetime(editor_df['rev_time']),
             _range = get_date_slider_from_datetime(editor_df['year_month']),
             granularity=Dropdown(options=['Yearly', 'Monthly', 'Daily'], value='Daily'),
             black=Dropdown(options=metrics, value='Conflict Score'),
             red=Dropdown(options= ['None'] + metrics, value='None'))

def select_editor(editor):
    global editor_df
    global the_editor
    global editor_inputname

    editor_inputname=editor
    
    wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
    try:
        the_editor = wikipedia_dv.get_editor(int(editor_inputname))
    except:
        the_editor = wikipedia_dv.get_editor(editor_inputname[2:])

    with out:
        %store the_editor
        %store editor_inputname

        clear_output()
        display(md("### Current Selection:"))
        if 'invalid' in the_editor:
            display(f"The editor {editor_inputname} was not found, try a different editor")
        else:
            # display the data that will be passed to the next notebook
            display(the_editor.to_frame('values'))
            display(md(f"#### Evolution of the Conflict Score of *{the_editor['name']}*"))

            editor_df = agg_actions[agg_actions['editor_id'] == the_editor['userid']].copy()
            #editor_df = calculator.elegible_actions[
                #calculator.elegible_actions['editor'] == editor_inputname].copy()

            display_conflict_score(editor_df)


def on_selection_change(change):

    try:
        select_editor(qg_obj.get_selected_df().iloc[0].name)
    except:
        print('Problem parsing the name. Execute the cell again and try a different editor.')

qgrid.set_grid_option('maxVisibleRows', 5)
qg_obj = qgrid.show_grid(editors_conflicts)
qg_obj.observe(on_selection_change, names=['_selected_rows'])
                       

display(md("### Select one editor (row) to continue the demo:"))
display(md('**Recomendation:** select an editor with *many conflicts* and *mid-high conflict score*'))
display(qg_obj)
out = Output()
display(out)

# select an editor that does not contain 0| at the beginning
for ed in editors_conflicts.index:
    if ed != 0:
        select_editor(ed)
        break


### Select one editor (row) to continue the demo:

**Recomendation:** select an editor with *many conflicts* and *mid-high conflict score*

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Output()

In the above graph you can select the *date range* and *granularity* (yearly, montly) 
of the timeline (X-axis), and plot any of the following counts in the black and red lines:
   
- **Conflict Score**: the sum of conflict scores of all actions divided by the number of elegible actions
- **Absolute Conflict Score**: the sum of conflict scores of all actions (without division)
- **Conflict Ratio**: the count of all conflicts divided by the number of elegible actions
- **Number of Conflicts**: the total number of conflicts
- **Total Elegible Actions**: the total number of elegible actions

### <span style="color:green"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU SELECT AN EDITOR </span>

In [99]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU SELECT AN EDITOR    ###
### ----------------------------------------------------------------------------------- ###

## This is the page you used ##

print('The page that is being used:', the_page['title'])

## Use the variable from the last notebook: conflicts_by_editors (pd.DataFrame)        ##
## Display the dataframe using interactive grid, you could learn more through the doc: ##
## https://qgrid.readthedocs.io/en/latest/                                             ##

qgrid.set_grid_option('maxVisibleRows', 5)
qgrid_init = qgrid.show_grid(conflicts_by_editors)
display(qgrid_init)

## Get the editor info with Wikipedia API (get_editor() method), more details you could check: ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/api.py                        ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/wikipedia.py                  ##

wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))

# This is an example editor index. You could change it manully by typing in a new index from
# the above grid, e.g. 692002
print('Editor id for example:', conflicts_by_editors.index[0])
editor_input_id = 296765
#store editor_input_id

# Get the editor's information in the form of pd.DataFrame
editor_info = wikipedia_dv.get_editor(int(editor_input_id))

#store editor_info

## Display the basic information of the selected editor ##

display(md("### Current Selection:"))
display(editor_info.to_frame('values'))

## Interactive evolution of conflict score of this editor, using ConflictListner, more details see ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/visualization/conflicts_listener.py        ##

display(md(f"#### Evolution of the Conflict Score of *{editor_info['name']}*"))

# Dataframe containing the info for interactive
editor_df = total_actions[total_actions['editor_id'] == editor_info['userid']].copy()
           
# Create a ConflictsListener instance.
conflicts_listener = ConflictsListener(editor_df)

# Set parameters
begin_date = date(2005, 3, 1)
end_date = date(2019, 6, 1)
frequency = 'Yearly' # 'Monthly', 'Daily'

# The metrics we need:
# ['Conflict Score', 'Absolute Conflict Score', 'Conflict Ratio', 'Number of Conflicts',
#  'Total Elegible Actions', ('None')]
# Note: only 'red_line' has 'None' option.
black_line = 'Conflict Score'
red_line = 'None'

conflicts_listener.listen(
    _range = (begin_date, end_date),
    granularity = frequency,
    black = black_line,
    red = red_line
)

The page that is being used: Random


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Editor id for example: 728997


### Current Selection:

,values
userid,296765
name,Wimt
editcount,31317
registration,2005-06-13T21:55:37Z
gender,male


#### Evolution of the Conflict Score of *Wimt*

AttributeError: Can only use .dt accessor with datetimelike values

In [5]:
def run_below(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Refresh the rest of the notebook below", button_style='info', min_width=500)
button.on_click(run_below)
display(button)

Button(button_style='info', description='Refresh the rest of the notebook below', style=ButtonStyle())

In [6]:
display(md("---"))
display(md(f"# B. Activity of editor on a page"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))
display(md("""In the following graph you can select the *date range* and *granularity* (yearly, montly) 
of the timeline (X-axis), and plot any of the follow counts in the black, red, blue and green lines:
   
- **adds**: number of first-time insertions
- **adds_surv_48h**: number of insertions for the first time that survived at least 48 hours
- **adds_persistent**:  number of insertions for the first time that survived until, at least, the end of the month
- **adds_stopword_count**:  number of insertions that were stop words
- **dels**: number of deletions
- **dels_surv_48h**: number of deletions that were not resinserted in the next 48 hours
- **dels_persistent**: number of deletions that were not resinserted until, at least, the end of the month
- **dels_stopword_count**: number of deletions that were stop words
- **reins**: number of reinsertions
- **reins_surv_48h**: number of reinsertionsthat survived at least 48 hours
- **reins_persistent**: number of reinsertionsthat survived until the end of the month
- **reins_stopword_count**: number of reinsertionsthat were stop words
"""))

---

# B. Activity of editor on a page

***Page: Random***

***Editor: SecretCal***

In the following graph you can select the *date range* and *granularity* (yearly, montly) 
of the timeline (X-axis), and plot any of the follow counts in the black, red, blue and green lines:
   
- **adds**: number of first-time insertions
- **adds_surv_48h**: number of insertions for the first time that survived at least 48 hours
- **adds_persistent**:  number of insertions for the first time that survived until, at least, the end of the month
- **adds_stopword_count**:  number of insertions that were stop words
- **dels**: number of deletions
- **dels_surv_48h**: number of deletions that were not resinserted in the next 48 hours
- **dels_persistent**: number of deletions that were not resinserted until, at least, the end of the month
- **dels_stopword_count**: number of deletions that were stop words
- **reins**: number of reinsertions
- **reins_surv_48h**: number of reinsertionsthat survived at least 48 hours
- **reins_persistent**: number of reinsertionsthat survived until the end of the month
- **reins_stopword_count**: number of reinsertionsthat were stop words


In [7]:
editor_agg_actions = agg_actions[agg_actions['editor_id']==the_editor.userid]

#Listener

listener = ActionsListener(editor_agg_actions)
actions = (editor_agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    editor_agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()

# Visualization
_range = get_date_slider_from_datetime(editor_agg_actions['year_month'])
interact(listener.listen, 
         _range = get_date_slider_from_datetime(editor_agg_actions['year_month']),
         editor=fixed('All'),
         granularity=Dropdown(options=['Yearly', 'Monthly'], value='Monthly'),
         black=Dropdown(options=actions, value='total'), 
         red=Dropdown(options= ['None'] + actions, value='total_surv_48h'),
         green=Dropdown(options= ['None'] + actions, value='None'), 
         blue=Dropdown(options= ['None'] + actions, value='None'))


interactive(children=(SelectionRangeSlider(continuous_update=False, description='Date Range', index=(0, 2), la…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [8]:
display(md("---"))
display(md(f"# C. Tokens that enter into conflict with other editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

display(md(""" The WordCloud displays the most common token strings (words) that a particular editor 
inserted or deleted and that enter into conflict with other editors. The size of the token string in 
the WordCloud indicates frequency of actions.

In the controls, you can select the *date range*, the type of *action* (insertion or deletion), and the 
*source*. The *source* can be any of the following:

-   **Only Conflicts**: use only the actions that are in conflict.
-   **Elegible Actions**: use only the actions that can potentially enter into conflict, i.e. actions 
that have occurred at least twice, e.g. the token x has been inserted twice (which necessarily implies 
it was remove once), the token x has been deleted twice (which necessarily implies it was inserted twice) 
-   **All Actions**: use all tokens regardles conflict
"""))

---

# C. Tokens that enter into conflict with other editors

***Page: Random***

***Editor: SecretCal***

 The WordCloud displays the most common token strings (words) that a particular editor 
inserted or deleted and that enter into conflict with other editors. The size of the token string in 
the WordCloud indicates frequency of actions.

In the controls, you can select the *date range*, the type of *action* (insertion or deletion), and the 
*source*. The *source* can be any of the following:

-   **Only Conflicts**: use only the actions that are in conflict.
-   **Elegible Actions**: use only the actions that can potentially enter into conflict, i.e. actions 
that have occurred at least twice, e.g. the token x has been inserted twice (which necessarily implies 
it was remove once), the token x has been deleted twice (which necessarily implies it was inserted twice) 
-   **All Actions**: use all tokens regardles conflict


In [9]:
sources = {
    'All actions': calculator.all_actions[calculator.all_actions['editor']==str(editor_inputname)],
    'Elegible Actions': calculator.elegible_actions[calculator.elegible_actions['editor']==str(editor_inputname)],
    'Only Conflicts': calculator.conflicts[calculator.conflicts['editor']==str(editor_inputname)],
}


# listener
listener = WCListener(sources)

# visualization
interact(listener.listen, 
         _range=get_date_slider_from_datetime(calculator.all_actions['rev_time']),
         source=Dropdown(options=list(listener.sources.keys()), value='Only Conflicts'),
         action=Dropdown(options=['Both', 'Just Insertions', 'Just Deletions'], value='Both'),
         editor=fixed('All'))

interactive(children=(SelectionRangeSlider(continuous_update=False, description='Date Range', index=(0, 37), l…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [10]:
display(md("---"))
display(md(f"# D. Tokens in the page owner by the editor"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))


display(md("""The following time line shows the token owned by this editor. The ownership 
(or authorship) is based in the WikiWho algorithm (
[Flöck & Acosta, 2014](http://wwwconference.org/proceedings/www2014/proceedings/p843.pdf)).
The graph shows that it is possible to recover the amount of tokens that an editor at any
point of time. The time points are selected based on instances in which insertions or deletions
were perfomerd in the editor's tokens. However, notice that the percentages of ownership might
vary because percentages are relative to  insertions or deletions of tokens of other editors.
This is why the current date is also included in the graph.

In the controls, you can select the *date range*, the *granularity* (Daily, Monthly, Yearly), and 
the *metric* that will be plotted (Tokens Owned or Tokens Owned(%)).
"""))

---

# D. Tokens in the page owner by the editor

***Page: Random***

***Editor: SecretCal***

The following time line shows the token owned by this editor. The ownership 
(or authorship) is based in the WikiWho algorithm (
[Flöck & Acosta, 2014](http://wwwconference.org/proceedings/www2014/proceedings/p843.pdf)).
The graph shows that it is possible to recover the amount of tokens that an editor at any
point of time. The time points are selected based on instances in which insertions or deletions
were perfomerd in the editor's tokens. However, notice that the percentages of ownership might
vary because percentages are relative to  insertions or deletions of tokens of other editors.
This is why the current date is also included in the graph.

In the controls, you can select the *date range*, the *granularity* (Daily, Monthly, Yearly), and 
the *metric* that will be plotted (Tokens Owned or Tokens Owned(%)).


In [11]:

all_actions = calculator.all_actions
listener = OwnedListener(all_actions, str(editor_inputname))
traces = ['Tokens Owned', 'Tokens Owned (%)']

# Visualization
interact(listener.listen,
         _range = get_date_slider_from_datetime(listener.days),
         granularity=Dropdown(options=['Yearly', 'Monthly', 'Daily'], value='Monthly'),
         trace=Dropdown(options=traces, value='Tokens Owned (%)', description='metric'))

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1266: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



interactive(children=(SelectionRangeSlider(continuous_update=False, description='Date Range', index=(0, 7), la…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [12]:
from utils.notebooks import get_next_notebook, get_previous_notebook

editor_actions = calculator.elegible_actions[calculator.elegible_actions['editor']==str(editor_inputname)]

if len(editor_actions) > 0:
    display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
else:
    display(HTML('<h3>This editor has no actions. Please select an editor that has '
                 'actions to continue to the next notebook.</h3>'))